In [9]:
import numpy
import getpass
import os
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_community.document_loaders import PyPDFLoader


In [2]:
# TODO it's good to do this to help trace what's going on inside the agent:

""" 
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

... or in the command line:

export LANGSMITH_TRACING="true"
export LANGSMITH_API_KEY="..."
"""

' \nimport getpass\nimport os\n\nos.environ["LANGSMITH_TRACING"] = "true"\nos.environ["LANGSMITH_API_KEY"] = getpass.getpass()\n\n... or in the command line:\n\nexport LANGSMITH_TRACING="true"\nexport LANGSMITH_API_KEY="..."\n'

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
# TODO use very terrible, cheap models instead of those SOTA models!

In [ ]:



if not os.environ.get("GOOGLE_API_KEY"):
  raise ValueError('no GOOGLE_API_KEY!')

from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [3]:

if not os.environ.get("GOOGLE_API_KEY"):
  raise ValueError('no GOOGLE_API_KEY!')

from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

In [4]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [11]:

# path = "/home/hugo/code/aria/assets/actuarial/5_Werner_Modlin.pdf"
path = "/home/hugo/code/aria/assets/actuarial/5_ASOP_12.pdf"

# Load and chunk contents of the blog
loader = PyPDFLoader(path) # TODO relative path
docs = loader.load()

# Chunk content
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

# Index chunks
_ = vector_store.add_documents(documents=all_splits)

# Define prompt for question-answering
# N.B. for non-US LangSmith endpoints, you may need to specify
# api_url="https://api.smith.langchain.com" in hub.pull.
prompt = hub.pull("rlm/rag-prompt")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [12]:
response = graph.invoke({"question": "Please give the exact definition of Adverse Selection according to ASOP 12"})
print(response["answer"])

I don't know the complete definition of Adverse Selection according to ASOP 12, as the provided text cuts off the definition mid-sentence. The available text states it is "Actions taken by one party using risk characteristics or other information known to or suspected by that party that cause a financial disadvantage to the". The context also notes that "antiselection" is considered synonymous with adverse selection.
